## Deep Learning - MVA - Simon Vittoz

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import pdb

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.layers import Conv2D
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.
/home/simon/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/simon/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/simon/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/simon/.local/lib/python3.6/site-packages/tensorflow/python/fra

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act return the action to execute, given a current state.
In evaluating mode it applies the learned policy.
In training mode it does not always apply the learned policy since it has to learn where would lead other policies.
Therefore we apply the learned policy with a probability $1 - \epsilon$ but there is still a $\epsilon$ probability that we act randomly. This allows to explore other possibilities. Without $\epsilon$ we would be stuck to one policy (one local optimum).

This is the __Trade-of Exploration-Exploitation__.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose - reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=4 # set small when debugging
epochs_test=4 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The board is the description of the environnement and where are the bonus and malus, it allows to get the reward after an action is performed. The position encode where the current state meaning the position of the rat in the grid.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        #perfom random action
        return(np.random.randint(0,4))

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        ##### FILL IN HERE
        state = env.reset()
        game_over = False
        win = 0
        lose = 0
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [22]:
import skvideo
skvideo.setFFmpegPath("/usr/bin")
import skvideo.io

# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.5/17.0. Average score (-6.5)
Win/lose count 10.0/16.0. Average score (-6.25)
Win/lose count 13.5/15.0. Average score (-4.666666666666667)
Win/lose count 10.0/9.0. Average score (-3.25)
Final score: -3.25


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p^\pi(.,.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

__Firt Step :__

By definition : 

$Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t=0}^{\infty}\gamma^{t}r(s_t,a_t)|s_0=s,a_0=a] = E_{p^{\pi}}[r(s,a) + \gamma \sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})|s_0=s,a_0=a]$

Since $s_0$ and $a_0$ are not random anymore, the law of $s_t$ and $a_t$ depends on $s_1$ and $a_1$ that we note $a'$ and $b'$. Therefor :

$Q^\pi(s,a) = E_{a',b' \sim p^{\pi}(.,.|s,a)}[r(s,a) + \gamma \sum_{t=0}^{\infty}\gamma^{t}r(s_{t+1},a_{t+1})] = E_{(s',a')\sim p^\pi(.,.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$ by definition.

__Second step :__

By definition : 

$Q^{\pi^*}(s,a) = max_{\pi} Q^{\pi}(s,a)$

A policy given, actions are not random anymore. Then $s \sim p^{\pi^*}(.|s,a) = \pi^*(.|s,a)$
Since $Q^{\pi^*}(s,a)$ is by definition a maximum, the action $a'$ will be chosen accordingly to this maximum.

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_a' Q^*(s',a')].
\end{equation*}

__Third step :__

Given the last equation we want $Q^*$ that satisfies :

$E_{s'\sim \pi^*(.|s,a)}([r(s,a)+\gamma\max_{a'}Q^{*}(s',a')] - Q^{*}(s,a)) = 0 .$

Therefore a good algorithm will decrease the value of this loss :

$L(\theta) = E_{s'\sim \pi^*(.|s,a)}||[r(s,a)+\gamma\max_{a'}Q(s',a',\theta)] - Q(s,a,\theta)||^2$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [8]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        #If we reach maximum size we pop an element
        if (len(self.memory) == self.max_memory):
            i = np.random.randint(len(self.memory))
            self.memory.pop(i)
        self.memory.append(m)

    def random_access(self):
        i = np.random.randint(len(self.memory))
        return(self.memory[i])

***
The pipeline we will use for training is given below:

In [9]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [10]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        pass

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            mem = self.memory.random_access()
            input_states[i] = np.array(mem[0])
            
            target_q[i] = self.model.predict(np.array([mem[0]]))
            
            if (mem[4] == True):
                target_q[i][mem[2]] = mem[3]
            else:
                target_q[i][mem[2]] = mem[3] + np.max(self.model.predict(np.array([mem[1]]))[0])

        ######## FILL INinfluence of
        
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.mEpoch 000/050 | Loss 0.0001 odel.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        opt = sgd(lr=lr, decay=1e-4, momentum=0.0)
                
        ####### FILL IN
        model = Sequential()
        model.add(Flatten())
        model.add(Dense(50,input_shape=(5*5*self.n_state,),activation='relu'))
        model.add(Dense(4,input_shape=(50,),activation='softmax'))
        
        model.compile(opt, "mse")
        self.model = model
    
    def learned_act(self, s):
        res = self.model.predict(np.array([s]))[0]
        return(np.argmax(res))

In [11]:
epochs_train = 50
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.01, epsilon = .1, memory_size=500, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))


Epoch 000/050 | Loss 0.0059 | Win/lose count 6.0/3.0 (3.0)
Epoch 001/050 | Loss 0.0072 | Win/lose count 2.5/4.0 (-1.5)
Epoch 002/050 | Loss 0.0142 | Win/lose count 5.0/3.0 (2.0)
Epoch 003/050 | Loss 0.0011 | Win/lose count 2.0/3.0 (-1.0)
Epoch 004/050 | Loss 0.0012 | Win/lose count 3.0/6.0 (-3.0)
Epoch 005/050 | Loss 0.0153 | Win/lose count 3.5/2.0 (1.5)
Epoch 006/050 | Loss 0.0047 | Win/lose count 7.0/6.0 (1.0)
Epoch 007/050 | Loss 0.0050 | Win/lose count 3.5/2.0 (1.5)
Epoch 008/050 | Loss 0.0017 | Win/lose count 3.5/5.0 (-1.5)
Epoch 009/050 | Loss 0.0109 | Win/lose count 4.5/3.0 (1.5)
Epoch 010/050 | Loss 0.0063 | Win/lose count 2.5/7.0 (-4.5)
Epoch 011/050 | Loss 0.0014 | Win/lose count 8.0/5.0 (3.0)
Epoch 012/050 | Loss 0.0100 | Win/lose count 2.5/3.0 (-0.5)
Epoch 013/050 | Loss 0.0202 | Win/lose count 2.5/2.0 (0.5)
Epoch 014/050 | Loss 0.0045 | Win/lose count 5.0/3.0 (2.0)
Epoch 015/050 | Loss 0.0102 | Win/lose count 3.5/1.0 (2.5)
Epoch 016/050 | Loss 0.0110 | Win/lose count 3.0/

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [30]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        opt = sgd(lr=lr, decay=1e-4, momentum=0.0)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(32, 3, activation='relu'))
        model.add(Conv2D(16, 3, activation='relu'))
        model.add(Flatten())
        model.add(Dense(4,activation='softmax'))
        
        model.compile(opt, "mse")
        self.model = model
        
    def learned_act(self, s):
        res = self.model.predict(np.array([s]))[0]
        return(np.argmax(res))

In [31]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=500, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0001 | Win/lose count 3.0/4.0 (-1.0)
Epoch 001/050 | Loss 0.0171 | Win/lose count 5.5/14.0 (-8.5)
Epoch 002/050 | Loss 0.0161 | Win/lose count 3.0/3.0 (0.0)
Epoch 003/050 | Loss 0.0190 | Win/lose count 3.0/5.0 (-2.0)
Epoch 004/050 | Loss 0.0095 | Win/lose count 5.5/5.0 (0.5)
Epoch 005/050 | Loss 0.0025 | Win/lose count 2.0/3.0 (-1.0)
Epoch 006/050 | Loss 0.0193 | Win/lose count 4.5/5.0 (-0.5)
Epoch 007/050 | Loss 0.0081 | Win/lose count 4.0/1.0 (3.0)
Epoch 008/050 | Loss 0.0023 | Win/lose count 5.0/2.0 (3.0)
Epoch 009/050 | Loss 0.0001 | Win/lose count 6.5/8.0 (-1.5)
Epoch 010/050 | Loss 0.0110 | Win/lose count 4.5/5.0 (-0.5)
Epoch 011/050 | Loss 0.0152 | Win/lose count 2.0/1.0 (1.0)
Epoch 012/050 | Loss 0.0034 | Win/lose count 3.5/0 (3.5)
Epoch 013/050 | Loss 0.0001 | Win/lose count 3.5/1.0 (2.5)
Epoch 014/050 | Loss 0.0054 | Win/lose count 3.5/2.0 (1.5)
Epoch 015/050 | Loss 0.0054 | Win/lose count 4.0/2.0 (2.0)
Epoch 016/050 | Loss 0.0016 | Win/lose count 2.0/1

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

On average the CNN algorithm performs better than the FC algorithm. This is expected since the map is a 2D image-like data for which CNN were specially designed.

We observe two problems in both algorithms :
- the agent often repeats the same moves and seems to be stuck in a small area.
- the agent does not explore all the map

Now let's have a look to the influence of temperature parameter.
When T is high (0.7) there is lot of rewards so the better algorithm will be the one which will explore the most, not taking into account malus. It is the FC algorithm.
When T is low (0.3) FC algorithm fall in all the mallus and therefor get a lower score than CNN.

In [59]:
epochs_test = 10

temperature = 0.7

env = Environment(grid_size=size, max_time=T,temperature=temperature)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon =  0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 10.0/1.0. Average score (9.0)
Win/lose count 7.0/3.0. Average score (6.5)
Win/lose count 3.5/3.0. Average score (4.5)
Win/lose count 2.5/3.0. Average score (3.25)
Win/lose count 4.5/1.0. Average score (3.3)
Win/lose count 7.5/4.0. Average score (3.3333333333333335)
Win/lose count 7.5/3.0. Average score (3.5)
Win/lose count 5.0/7.0. Average score (2.8125)
Win/lose count 6.0/3.0. Average score (2.8333333333333335)
Win/lose count 4.5/4.0. Average score (2.6)
Final score: 2.6
Test of the FC
Win/lose count 5.5/1.0. Average score (4.5)
Win/lose count 7.0/3.0. Average score (4.25)
Win/lose count 2.5/2.0. Average score (3.0)
Win/lose count 5.0/4.0. Average score (2.5)
Win/lose count 11.0/1.0. Average score (4.0)
Win/lose count 13.5/3.0. Average score (5.083333333333333)
Win/lose count 8.0/4.0. Average score (4.928571428571429)
Win/lose count 6.0/4.0. Average score (4.5625)
Win/lose count 6.5/6.0. Average score (4.111111111111111)
Win/lose count 6.5/5.0. Average s

In [58]:
epochs_test = 10

temperature = 0.3

env = Environment(grid_size=size, max_time=T,temperature=temperature)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon =  0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 4.5/4.0. Average score (0.5)
Win/lose count 3.0/4.0. Average score (-0.25)
Win/lose count 2.0/1.0. Average score (0.16666666666666666)
Win/lose count 2.0/2.0. Average score (0.125)
Win/lose count 1.0/1.0. Average score (0.1)
Win/lose count 2.5/0. Average score (0.5)
Win/lose count 2.0/1.0. Average score (0.5714285714285714)
Win/lose count 3.0/0. Average score (0.875)
Win/lose count 1.5/1.0. Average score (0.8333333333333334)
Win/lose count 3.0/3.0. Average score (0.75)
Final score: 0.75
Test of the FC
Win/lose count 2.0/4.0. Average score (-2.0)
Win/lose count 4.5/3.0. Average score (-0.25)
Win/lose count 5.5/5.0. Average score (0.0)
Win/lose count 3.0/1.0. Average score (0.5)
Win/lose count 4.5/6.0. Average score (0.1)
Win/lose count 4.0/3.0. Average score (0.25)
Win/lose count 5.0/3.0. Average score (0.5)
Win/lose count 5.5/5.0. Average score (0.5)
Win/lose count 4.0/7.0. Average score (0.1111111111111111)
Win/lose count 2.5/5.0. Average score (-0.15)
F

In [43]:
HTML(display_videos('cnn_test10.mp4'))

FileNotFoundError: [Errno 2] No such file or directory: 'cnn_test10.mp4'

In [40]:
HTML(display_videos('fc_test10.mp4'))

FileNotFoundError: [Errno 2] No such file or directory: 'fc_test10.mp4'

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.

__First step :__

We try the first idea. This seems to work well ! The score improve from 0.75 to 3.55.

In [40]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    EPS = np.ones(epoch)
    for i in range(10):
        EPS[i*epoch//10:(i+1)*epoch//10] = (1 - i/10)
        

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        agent.set_epsilon(EPS[e])

        while not game_over:
            # The agent performs an action
            
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
# Training
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=500, batch_size = 32,n_state=2)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0209 | Win/lose count 8.0/14.0 (-6.0)
Epoch 001/050 | Loss 0.0187 | Win/lose count 8.0/12.0 (-4.0)
Epoch 002/050 | Loss 0.0354 | Win/lose count 12.5/16.0 (-3.5)
Epoch 003/050 | Loss 0.0358 | Win/lose count 11.5/18.0 (-6.5)
Epoch 004/050 | Loss 0.0113 | Win/lose count 7.0/9.0 (-2.0)
Epoch 005/050 | Loss 0.0269 | Win/lose count 10.5/18.0 (-7.5)
Epoch 006/050 | Loss 0.0169 | Win/lose count 12.5/9.0 (3.5)
Epoch 007/050 | Loss 0.0063 | Win/lose count 9.0/15.0 (-6.0)
Epoch 008/050 | Loss 0.0132 | Win/lose count 9.0/9.0 (0.0)
Epoch 009/050 | Loss 0.0042 | Win/lose count 4.5/13.0 (-8.5)
Epoch 010/050 | Loss 0.0197 | Win/lose count 13.0/13.0 (0.0)
Epoch 011/050 | Loss 0.0246 | Win/lose count 11.0/6.0 (5.0)
Epoch 012/050 | Loss 0.0126 | Win/lose count 15.5/11.0 (4.5)
Epoch 013/050 | Loss 0.0233 | Win/lose count 16.5/16.0 (0.5)
Epoch 014/050 | Loss 0.0068 | Win/lose count 7.0/13.0 (-6.0)
Epoch 015/050 | Loss 0.0148 | Win/lose count 8.0/4.0 (4.0)
Epoch 016/050 | Loss 0.0242 

In [41]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_train_exploremodel.h5',name_model='cnn_train_exploremodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')

Test of the CNN
Win/lose count 6.0/5.0. Average score (1.0)
Win/lose count 13.5/2.0. Average score (6.25)
Win/lose count 5.0/1.0. Average score (5.5)
Win/lose count 7.5/4.0. Average score (5.0)
Win/lose count 9.0/9.0. Average score (4.0)
Win/lose count 1.5/0. Average score (3.5833333333333335)
Win/lose count 8.0/3.0. Average score (3.7857142857142856)
Win/lose count 6.5/8.0. Average score (3.125)
Win/lose count 7.5/4.0. Average score (3.1666666666666665)
Win/lose count 13.0/6.0. Average score (3.55)
Final score: 3.55


__Second Step__ :

Now we try the second idea. This does not seem to work well, the score decrease a lot. Maybe I did not understand what was intented.

In [52]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""
        

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
            
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        
        self.t = self.t + 1
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0
        
        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state
    
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=500, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore_2')

Epoch 000/050 | Loss 0.0445 | Win/lose count 10.5/31.500000000000064 (-21.000000000000064)
Epoch 001/050 | Loss 0.0279 | Win/lose count 10.700000000000003/34.300000000000104 (-23.6000000000001)
Epoch 002/050 | Loss 0.0182 | Win/lose count 8.900000000000004/33.70000000000011 (-24.800000000000104)
Epoch 003/050 | Loss 0.0109 | Win/lose count 12.400000000000006/36.70000000000014 (-24.300000000000132)
Epoch 004/050 | Loss 0.0333 | Win/lose count 5.6000000000000005/29.500000000000128 (-23.900000000000126)
Epoch 005/050 | Loss 0.0328 | Win/lose count 9.200000000000003/31.500000000000124 (-22.30000000000012)
Epoch 006/050 | Loss 0.0110 | Win/lose count 8.500000000000002/35.00000000000014 (-26.500000000000142)
Epoch 007/050 | Loss 0.0215 | Win/lose count 8.300000000000002/28.800000000000107 (-20.500000000000107)
Epoch 008/050 | Loss 0.0194 | Win/lose count 8.000000000000002/28.900000000000095 (-20.90000000000009)
Epoch 009/050 | Loss 0.0050 | Win/lose count 13.300000000000006/30.80000000000010

In [53]:
env = EnvironmentExploring(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=500, batch_size = 32)
agent_cnn.load(name_weights='cnn_train_explore_2model.h5',name_model='cnn_train_explore_2model.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')

Test of the CNN
Win/lose count 5.0/21.500000000000032. Average score (-16.500000000000032)
Win/lose count 11.5/20.40000000000003. Average score (-12.700000000000031)
Win/lose count 9.600000000000003/19.800000000000004. Average score (-11.866666666666688)
Win/lose count 4.6/22.10000000000005. Average score (-13.275000000000029)
Win/lose count 10.700000000000001/21.60000000000003. Average score (-12.80000000000003)
Win/lose count 12.500000000000005/22.100000000000026. Average score (-12.266666666666694)
Win/lose count 15.700000000000008/24.900000000000077. Average score (-11.828571428571463)
Win/lose count 14.000000000000007/18.900000000000006. Average score (-10.96250000000003)
Win/lose count 11.400000000000006/22.00000000000004. Average score (-10.922222222222253)
Win/lose count 10.800000000000002/20.30000000000003. Average score (-10.780000000000031)
Final score: -10.780000000000031


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***